In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import koreanize_matplotlib
from datetime import datetime, timedelta


### 시장별 품목 확인
#### 시장 선정 - 등록된 가게가 많은 곳
데이터 구조: 지역 < 시장(market) < 가게(store)

In [ ]:
file_path = "../data/raw/ikh_store_meta_2025-01-13.csv"
df = pd.read_csv(file_path, dtype={"store_id": str})
df


In [ ]:
df_ref = df[~df.duplicated(subset=["store_id"])]  # 중복 제거
df_ref = df_ref[df_ref["comments"] >= 100]
market_counts = df_ref.groupby("market").count()["store_id"]

# count 값이 5 이상인 market 필터링
markets = market_counts[market_counts >= 5].sort_values(ascending=False)
# print(markets, '\ntotal:', markets.sum())
df_ftd = df_ref[df_ref["market"].isin(markets.index)]



In [ ]:
plt.figure(figsize=(10, 6))
markets.plot(kind="bar", color="skyblue", edgecolor="black")
plt.title("시장별 조건을 만족하는 가게 수")
plt.xlabel("시장")
plt.ylabel("가게 수")
plt.xticks(rotation=45)
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()

# Markets 데이터 총합 출력
print("총 가게 수:", markets.sum())

시장 선정 - 리뷰가 100건 이상 등록된 가게가 5개 이상인 시장
(리뷰가 많을수록 오래되고, 데이터가 많고, 인어교주해적단 가격 시스템을 잘 이해하고 있을 것이라고 가정)

- 노량진 1층
- 가락시장
- 노량진 2층
- 구리농수산물시장
- 강서농수산물시장
- 수원농수산물시장
- 안양평촌농수산물시장
- 인천종합연안부두어시장
- 소래포구종합어시장
- 마포농수산물시장
- 부산민락어민활어직판장

-> 11개 시장 153 가게 대상

todo: 해당 가게들에서 주로 파는 품목 확인

In [ ]:
import time
today = time.strftime("%Y-%m-%d")
output_file = f"../data/processed/ikh_store_meta_{today}.csv"
df_ftd.to_csv(output_file, index=False, encoding="utf-8")

#### 품목 선정 - 선정된 가게에서 주로 판매하는 품목

In [ ]:
file_path = "../data/raw/ikh_store_item_2025-01-14.csv"
df = pd.read_csv(file_path, dtype={"store_id": str})
df.head()
# TODO: baseId, labelId, itemCategoryId 의 의미 파악 필요

### 가격 변동 확인
품목별 데이터가 얼마나 있는지를 확인한 후 품목 결정

In [ ]:
# 컬럼별 고유 항목 수 확인 및 시각화
columns = ['store', 'itemId', 'baseId', 'labelId', 'itemName', 'labelOrderNo',
           'stateDesc', 'raisedDesc', 'originDesc', 'regionDesc', 'unitDesc', 'sizeDesc']
# for column in df.columns:
for column in columns:
    # 항목별 개수 계산
    value_counts = df[column].value_counts()
    
    # Value Count가 10 이상인 항목만 필터링
    value_counts_filtered = value_counts[value_counts >= 10]
    
    # 조건을 만족하는 항목이 있을 때만 시각화
    if not value_counts_filtered.empty:
    # if not value_counts.empty:
        plt.figure(figsize=(30, 3))
        value_counts_filtered.plot(kind="bar", color="skyblue", edgecolor="black")
        plt.title(f"Column: {column} - Value Counts (>= 10)")
        # value_counts.plot(kind="bar", color="skyblue", edgecolor="black")        
        # plt.title(f"Column: {column} - Value Counts")
        plt.xlabel("Unique Values")
        plt.ylabel("Counts")
        plt.xticks(rotation=45, ha="right")
        plt.tight_layout()
        plt.show()

In [ ]:
# 품목별 판매중인 가게 수 확인
# itemName별 store_id의 고유 개수 계산
item_store_count = df.groupby("itemName")["store_id"].nunique()

item_store_count_df = item_store_count.reset_index()
item_store_count_df.columns = ["itemName", "store_count"]

item_store_count_sorted = item_store_count.sort_values(ascending=False)
item_store_count_sorted.plot(kind="bar", figsize=(30, 3), color="skyblue", edgecolor="black")
plt.title("Number of Stores Selling Each Item")
plt.xlabel("Item Name")
plt.ylabel("Number of Stores")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

item_store_count_sorted[:20].plot(kind="bar", figsize=(10, 3), color="skyblue", edgecolor="black")
plt.title("Number of Stores Selling Each Item top 20")
plt.xlabel("Item Name")
plt.ylabel("Number of Stores")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()


In [ ]:
# 품목별 세부품목 수 확인
top_20_item_names = item_store_count_sorted[:20].index

# 상위 20개 itemName에 대한 데이터 필터링
filtered_df = df[df["itemName"].isin(top_20_item_names)]

# itemId별 count와 itemName 정보 확인
item_id_counts = filtered_df.groupby(["itemName", "itemId"]).size().reset_index(name="count")

# itemName-itemId 형식으로 라벨 생성
item_id_counts["label"] = item_id_counts["itemName"] + "-" + item_id_counts["itemId"].astype(str)

item_id_counts = item_id_counts[item_id_counts["count"] >= 20]
# 시각화
plt.figure(figsize=(30, 6))
plt.bar(item_id_counts["label"], item_id_counts["count"], color="skyblue", edgecolor="black")
plt.title("Counts of ItemId by ItemName")
plt.xlabel("ItemName-ItemId")
plt.ylabel("Count")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

item_id_counts = item_id_counts.sort_values(by="count", ascending=False)
plt.figure(figsize=(30, 6))
plt.bar(item_id_counts["label"], item_id_counts["count"], color="skyblue", edgecolor="black")
plt.title("Counts of ItemId by ItemName")
plt.xlabel("ItemName-ItemId")
plt.ylabel("Count")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()




기존: 광어, 방어, 연어, 참돔, 대게, 가리비
변경: 농어(53), 광어(3), 대게(17), 연어(100), 우럭(5906), 참돔(54), 방어(3849)

In [ ]:
file_path = "../data/raw/ikh_item_price_2025-01-14.csv"
df = pd.read_csv(file_path)
df["priceDate"] = pd.to_datetime(df["priceDate"])

df.head()

In [ ]:
# df[df["item"] == "연어"]

In [ ]:
# 그룹화하여 아이템별, 날짜별 평균 가격 계산
avg_price_by_item_date = df.groupby(["item", "priceDate"])["avgPrice"].mean().reset_index()

# 아이템별 시각화
plt.figure(figsize=(12, 6))
for item in avg_price_by_item_date["item"].unique():
    item_data = avg_price_by_item_date[avg_price_by_item_date["item"] == item]
    plt.plot(item_data["priceDate"], item_data["avgPrice"], label=item)

# 그래프 꾸미기
plt.title("Average Price by Date for Each Item")
plt.xlabel("Date")
plt.ylabel("Average Price")
plt.legend(title="Item")
plt.grid()
plt.tight_layout()
plt.show()


In [ ]:
# 평균 가격 계산
avg_price_by_item_market = df.groupby(["item", "market"])["avgPrice"].mean().reset_index()

# 아이템별 시장별 평균 가격 시각화
plt.figure(figsize=(14, 8))
sns.barplot(
    x="market", 
    y="avgPrice", 
    hue="item", 
    data=avg_price_by_item_market,
    estimator=lambda x: x.mean(),
    dodge=True
)

# 그래프 꾸미기
plt.title("Average Price by Market and Item")
plt.xlabel("Market")
plt.ylabel("Average Price")
plt.xticks(rotation=45, ha="right")
plt.legend(title="Item")
plt.tight_layout()
plt.show()



# 현재 날짜 기준으로 최근 1년 데이터 필터링
one_year_ago = datetime.now() - timedelta(days=365)
df["priceDate"] = pd.to_datetime(df["priceDate"])  # priceDate를 datetime 형식으로 변환
recent_data = df[df["priceDate"] >= one_year_ago]

# 평균 가격 계산 (최근 1년 데이터 사용)
avg_price_by_item_market = recent_data.groupby(["item", "market"])["avgPrice"].mean().reset_index()

# 아이템별 시장별 평균 가격 시각화
plt.figure(figsize=(14, 8))
sns.barplot(
    x="market", 
    y="avgPrice", 
    hue="item", 
    data=avg_price_by_item_market,
    estimator=lambda x: x.mean(),
    dodge=True
)

# 그래프 꾸미기
plt.title("Average Price by Market and Item (Last 1 Year)")
plt.xlabel("Market")
plt.ylabel("Average Price")
plt.xticks(rotation=45, ha="right")
plt.legend(title="Item")
plt.tight_layout()
plt.show()

In [ ]:
# # 시장별 평균 가격 비교
# plt.figure(figsize=(12, 6))
# sns.barplot(x="market", y="avgPrice", data=df, estimator=lambda x: x.mean())
# plt.title("Average Price by Market")
# plt.xlabel("Market")
# plt.ylabel("Average Price")
# plt.xticks(rotation=45)
# plt.show()

In [ ]:
# 품목별 가격 변동성 확인
price_variation = df.groupby("item")["avgPrice"].std()

plt.figure(figsize=(10, 6))
price_variation.plot(kind="bar", title="Price Variation by Item")
plt.ylabel("Standard Deviation of Price")
plt.xlabel("Item")
plt.xticks(rotation=45)
plt.grid()
plt.show()

In [ ]:
# 품목별 평균 가격 계산
item_price_trend = df.groupby(["priceDate", "item"])["avgPrice"].mean().unstack()

# 품목별 시간에 따른 가격 변화 플롯
plt.figure(figsize=(14, 8))
for column in item_price_trend.columns:
    plt.plot(item_price_trend.index, item_price_trend[column], label=column)

plt.title("Average Price Trends by Item", fontsize=16)
plt.xlabel("Date", fontsize=12)
plt.ylabel("Average Price", fontsize=12)
plt.legend(title="Item", fontsize=10)
plt.grid()
plt.show()

In [ ]:
# 전체 날짜 범위 생성
date_range = pd.date_range(start=df["priceDate"].min(), end=df["priceDate"].max())
print(f"전체날짜: {len(date_range)}")

# 품목별 누락된 날짜 찾기
missing_dates = {}

for item in df["item"].unique():
    # 해당 품목의 날짜 데이터
    item_dates = df[df["item"] == item]["priceDate"]
    
    # 누락된 날짜 계산
    missing_dates[item] = date_range.difference(item_dates)

# 누락된 날짜 출력
for item, dates in missing_dates.items():
    print(f"품목: {item} / \t누락된 날짜 수: {len(dates)}")


In [ ]:
print(f"전체날짜: {len(date_range)}")

# 품목-시장별 누락된 날짜 계산
missing_data = []
for (item, market), group in df.groupby(["item", "market"]):
    # 해당 품목-시장의 날짜 데이터
    item_market_dates = group["priceDate"]
    # 누락된 날짜 계산
    missing_dates = date_range.difference(item_market_dates)
    missing_count = len(missing_dates)
    
    # 결과 저장
    missing_data.append({"Item": item, "Market": market, "Missing Count": missing_count})
    
    # 누락된 날짜 출력
    print(f"품목: {item}, 시장: {market} / \t누락된 날짜 수: {missing_count}")


# DataFrame 생성
missing_df = pd.DataFrame(missing_data)


- 우럭: 23년 이전 데이터가 없음
- 방어: 20년 이전 데이터가 없음 + 여름 데이터 없음
todo: 품목의 시세가 메뉴의 가격에 영향을 주는지 확인